# Configuration

Project configuration is stored in `jurassic.json` file

In [1]:
//| export

import { z } from "zod";
import path from "node:path";
import { getNotebooksToProcess } from "jurassic/utils.ts";

In [2]:
//| export

const configSchema: z.Schema = z.object({
  configPath: z.string(),
  nbsPath: z.string().default("."),
  outputPath: z.string().default("."),
  docsInputPath: z.string().default("."),
  docsOutputPath: z.string().default("."),
  vitepress: z.object({
    title: z.string(),
    description: z.string(),
    base: z.string(),
    cleanUrls: z.boolean().default(true),
    themeConfig: z.object({
      nav: z.array(
        z.object({
          text: z.string(),
          link: z.string(),
        }),
      ),
      search: z.object({
        provider: z.string(),
      }),
      sidebar: z.array(
        z.object({
          text: z.string(),
          items: z.array(
            z.object({
              text: z.string(),
              link: z.string(),
            }),
          ),
        }),
      ),
      socialLinks: z.array(
        z.object({
          icon: z.string(),
          link: z.string(),
        }),
      ),
    }),
  }),
}).transform((data) => {
  return Object.assign(data, {
    notebooks: getNotebooksToProcess(".", data.nbsPath),
  });
});

export type Config = z.infer<typeof configSchema>;

🕵️‍♀️ Looking for config. Start from `cwd` and keep going up if needed looking
for `jurassic.json`. When running notebooks, it seems like `cwd` points to
notebook's directory (at least when running in VS Code). Hence this extra
gymnastics, just to be on the safe side. Notice `d` (depth) and `maxD` (max
depth) to make sure things don't get out of control

In [3]:
//| export

const findConfig = async (
  dir: string = Deno.cwd(),
  d = 0,
  config = "jurassic.json",
  maxD = 10,
): Promise<string> => {
  if (d >= maxD) throw new Error("max depth reached");

  try {
    const f = path.join(dir, config);
    await Deno.lstat(f);
    return f;
  } catch {
    return findConfig(path.join(dir, "../"), d + 1);
  }
};

In [4]:
await findConfig();

"/Users/philip/projects/jurassic/jurassic.json"

In [5]:
//| export

export const getTestConfig = (baseDir: string): Config =>
  configSchema.parse({
    configPath: path.resolve(baseDir, "jurassic.json"),
    nbsPath: path.resolve(baseDir, "nbs"),
    outputPath: path.resolve(baseDir, "jurassic"),
    docsInputPath: path.resolve(baseDir, "docs"),
    docsOutputPath: path.resolve(baseDir, "_docs"),
    vitepress: {
      title: "Jurassic",
      description: "Jurassic docs",
      base: "/jurassic/",
      themeConfig: {
        nav: [{ text: "Home", link: "/" }],
        search: {
          provider: "local",
        },
        sidebar: [],
        socialLinks: [
          { icon: "github", link: "https://github.com/callmephilip/jurassic" },
        ],
      },
    },
  });

In [6]:
getTestConfig("../");

{
  configPath: "/Users/philip/projects/jurassic/jurassic.json",
  nbsPath: "/Users/philip/projects/jurassic/nbs",
  outputPath: "/Users/philip/projects/jurassic/jurassic",
  docsInputPath: "/Users/philip/projects/jurassic/docs",
  docsOutputPath: "/Users/philip/projects/jurassic/_docs",
  vitepress: {
    title: "Jurassic",
    description: "Jurassic docs",
    base: "/jurassic/",
    cleanUrls: true,
    themeConfig: {
      nav: [ { text: "Home", link: "/" } ],
      search: { provider: "local" },
      sidebar: [],
      socialLinks: [
        {
          icon: "github",
          link: "https://github.com/callmephilip/jurassic"
        }
      ]
    }
  },
  notebooks: [
    "utils.ipynb",
    "submodule/hello.ipynb",
    "docs.ipynb",
    "notebooks.ipynb",
    "config.ipynb",
    "export.ipynb"
  ]
}

Load and parse config.

In [7]:
//| export

export const getConfig = async (): Promise<Config> => {
  const cf = await findConfig();
  const dcf = path.dirname(cf);
  const c = configSchema.parse(
    Object.assign({ configPath: cf }, JSON.parse(await Deno.readTextFile(cf))),
  );
  c.nbsPath = path.join(dcf, c.nbsPath);
  c.outputPath = path.join(dcf, c.outputPath);
  c.docsInputPath = path.join(dcf, c.docsInputPath);
  c.docsOutputPath = path.join(dcf, c.docsOutputPath);
  return c;
};